In [9]:
from ultralytics import YOLO

# YOLOv8 모델 불러오기 (pre-trained 모델)
model = YOLO("../runs/detect/train/weights/best.pt")

# YOLO 모델을 통해 탐지 결과 가져오기
results = model("./Guppy_fish_dataset_YOLO/train/images/ikan_v4_002.jpg")

# 첫 번째 결과 객체 접근
result = results[0]

# 바운딩 박스를 가져오는 방법
boxes = result.boxes  # Boxes 객체로 반환됩니다.

# 바운딩 박스 좌표(x_center, y_center, width, height) 추출
xywh = boxes.xywh.cpu().numpy()  # (x_center, y_center, width, height)

# 바운딩 박스 출력
for box in xywh:
    x_center, y_center, w, h = box
    print(f"Bounding Box: x={x_center}, y={y_center}, w={w}, h={h}")


image 1/1 c:\Source\YOLO_Practice\YOLO_V8\Guppy_fish_dataset_YOLO\train\images\ikan_v4_002.jpg: 384x640 4 guppys, 151.3ms
Speed: 2.1ms preprocess, 151.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Bounding Box: x=1809.908935546875, y=200.94061279296875, w=150.456298828125, h=124.20698547363281
Bounding Box: x=1584.956787109375, y=142.5042724609375, w=176.4591064453125, h=127.42100524902344
Bounding Box: x=1531.294921875, y=204.8870849609375, w=65.268310546875, h=135.3074188232422
Bounding Box: x=548.2476806640625, y=151.1898956298828, w=153.27420043945312, h=99.24712371826172


In [10]:
import torch
import torchvision.models as models
import torch.nn as nn

class FishReIDModel(nn.Module):
    def __init__(self, backbone=models.resnet50(pretrained=True), embedding_size=512):
        super(FishReIDModel, self).__init__()
        
        # ResNet 백본 네트워크 (사전 학습된 모델 사용)
        self.backbone = backbone
        
        # 마지막 레이어는 임베딩 차원으로 변환하는 Fully Connected Layer
        self.fc = nn.Linear(2048, embedding_size)
        
    def forward(self, x):
        x = self.backbone(x)  # 특징 추출
        x = x.view(x.size(0), -1)  # 배치 크기 맞추기
        x = self.fc(x)  # 임베딩으로 변환
        return x

In [13]:
import torch
from ultralytics import YOLO
from torchvision import transforms
from PIL import Image
import numpy as np

# YOLOv8 모델 불러오기 (사전 학습된 모델)
yolo_model = YOLO("../runs/detect/train/weights/best.pt")
reid_model = FishReIDModel()  # 앞서 정의한 Re-ID 모델 (ResNet 기반)

# Re-ID 모델을 평가 모드로 설정
reid_model.eval()

# 이미지 로드 및 전처리
image_path = "./Guppy_fish_dataset_YOLO/train/images/ikan_v4_002.jpg"
img = Image.open(image_path)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_img = transform(img).unsqueeze(0)

# 첫 번째 예측 결과를 가져옵니다.
result = results[0]

# 바운딩 박스를 가져옵니다.
boxes = result.boxes

# 바운딩 박스 좌표 (x_center, y_center, width, height) 추출
xywh = boxes.xywh.cpu().numpy()  # (x_center, y_center, width, height) 형태로 변환

# 각 물고기의 특징 추출
for box in boxes:
    x_center, y_center, w, h = box
    # 바운딩 박스를 이용해 물고기 부분 자르기
    crop_img = img.crop((x_center - w / 2, y_center - h / 2, x_center + w / 2, y_center + h / 2))
    crop_img = transform(crop_img).unsqueeze(0)  # 크롭된 이미지를 Re-ID 모델로 입력
    
    # Re-ID 모델을 통해 물고기의 임베딩 벡터 추출
    with torch.no_grad():
        embedding = reid_model(crop_img)
    
    # 여기서 embedding을 저장하거나, 다른 물고기 이미지와 유사도 계산 후, 같은 물고기인지 다른 물고기인지 구별합니다.
    print(f"Embeddings for detected fish: {embedding}")

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# 두 물고기의 임베딩 벡터 비교 (Cosine Similarity)
similarity = cosine_similarity(embedding1, embedding2)
print(f"Similarity between the two fish: {similarity}")

ModuleNotFoundError: No module named 'sklearn'